In [7]:
!pip install spektral==0.6.2 #feel free to use any other library of your choice
!pip install keras
!pip install tensorflow

     |████████████████████████████████| 95 kB 2.8 MB/s 
  Attempting uninstall: spektral
    Found existing installation: spektral 0.6.0
    Uninstalling spektral-0.6.0:
      Successfully uninstalled spektral-0.6.0


In [8]:
import numpy as np
import tensorflow as tf
import spektral
tf.random.set_seed(9)

In [9]:
adj, features,labels, train_mask, validation_mask, test_mask = spektral.datasets.citation.load_data(dataset_name='cora')

Loading cora dataset
Pre-processing node features


In [10]:
#convert feature from sparse to dense 
features = features.todense()
adj = adj.todense() + np.eye(adj.shape[0])
# features = tf.convert_to_tensor(features, dtype=tf.float32)
# adj = tf.convert_to_tensor(adj, dtype=tf.float32)
features = features.astype('float32')
adj = adj.astype('float32')

In [11]:
print("No of training  dataset = ",np.sum(train_mask))
print("No of validation  dataset = ",np.sum(validation_mask))
print("No of test  dataset = ",np.sum(test_mask))
print("No of features = ",features.shape[1])

No of training  dataset =  140
No of validation  dataset =  500
No of test  dataset =  1000
No of features =  1433


In [12]:
#helper functions 
def compute_masked_cross_entropy_loss(predicted,labels,mask):
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=predicted, labels=labels)
    mask = tf.cast(mask,dtype=tf.float32)
    mask = mask / tf.reduce_mean(mask)
    loss = loss * mask
    return tf.reduce_mean(loss)
def compute_masked_accuracy(predicted,gt_label,mask):
    correct_predicted = tf.equal(tf.argmax(predicted,axis= 1),tf.argmax(gt_label,axis= 1))
    correct_predicted = tf.cast(correct_predicted,dtype= tf.float32)
    mask = tf.cast(mask,dtype= tf.float32)
    mask = mask / tf.reduce_mean(mask)
    correct_predicted = correct_predicted * mask
    return tf.reduce_mean(correct_predicted)

In [13]:
def gnn(features, adj, transform, activation):
    int_feature = transform(features)
    segregrate_feature = tf.matmul(adj,int_feature)
    return activation(segregrate_feature)

In [14]:
from tensorflow.python.keras import optimizers
def train(features,adjacency,gnn_fn,units,epochs,lr):
    lyr_1 = tf.keras.layers.Dense(units)
    lyr_2 = tf.keras.layers.Dense(7)
    def cora_gnn(features,adjacency):
        hidden_layer1 = gnn_fn(features,adjacency,lyr_1,tf.nn.relu)
        output = gnn_fn(hidden_layer1,adjacency,lyr_2,tf.identity)
        return output
    optim = tf.keras.optimizers.Adam(learning_rate=lr)
    best_score = 0.0
    for epoch in range(0,epochs):
        with tf.GradientTape() as tg:
            predicted = cora_gnn(features,adjacency)
            loss = compute_masked_cross_entropy_loss(predicted,labels,train_mask)
        variables = tg.watched_variables()
        grads = tg.gradient(loss,variables)
        optim.apply_gradients(zip(grads,variables))
        predicted = cora_gnn(features,adjacency)
        val_score = compute_masked_accuracy(predicted,labels,validation_mask)
        if(val_score > best_score):
            best_score = val_score
            test_score = compute_masked_accuracy(predicted,labels,test_mask)
            print("Epoch = ",epoch," Train loss = ",loss.numpy()," Validation accuracy = ",val_score.numpy()
                  , " Test Accuracy = ",test_score.numpy())

In [15]:
train(features,adj,gnn,32,200,0.01)

Epoch =  0  Train loss =  2.0721896  Validation accuracy =  0.22399999  Test Accuracy =  0.24599999
Epoch =  1  Train loss =  1.7766834  Validation accuracy =  0.42599997  Test Accuracy =  0.44899994
Epoch =  3  Train loss =  1.5689648  Validation accuracy =  0.518  Test Accuracy =  0.503
Epoch =  4  Train loss =  1.395454  Validation accuracy =  0.608  Test Accuracy =  0.593
Epoch =  5  Train loss =  1.2508626  Validation accuracy =  0.678  Test Accuracy =  0.68299997
Epoch =  6  Train loss =  1.12273  Validation accuracy =  0.714  Test Accuracy =  0.74499995
Epoch =  7  Train loss =  1.0080339  Validation accuracy =  0.724  Test Accuracy =  0.76400006
Epoch =  8  Train loss =  0.9256463  Validation accuracy =  0.73199993  Test Accuracy =  0.77000004
Epoch =  9  Train loss =  0.8671585  Validation accuracy =  0.73999995  Test Accuracy =  0.779
Epoch =  10  Train loss =  0.7925489  Validation accuracy =  0.75999993  Test Accuracy =  0.79099995
Epoch =  11  Train loss =  0.72351336  Val

In [17]:
#mean pooling gcc 
# adj = tf.cast(adj,dtype=tf.float32)
# deg = tf.reduce_mean(adj,axis =-1)
# deg = tf.cast(deg,dtype=tf.float32)
# mean_adj = tf.math.divide(adj, deg)
deg = tf.reduce_mean(adj,axis =-1)
# mean_adj = tf.math.divide(adj, deg)
train(features,adj/deg,gnn,32,200,0.01)

Epoch =  0  Train loss =  42869.117  Validation accuracy =  0.156  Test Accuracy =  0.154
Epoch =  1  Train loss =  77384.07  Validation accuracy =  0.232  Test Accuracy =  0.235
Epoch =  2  Train loss =  68750.34  Validation accuracy =  0.278  Test Accuracy =  0.25699997
Epoch =  3  Train loss =  58334.027  Validation accuracy =  0.38199997  Test Accuracy =  0.38999996
Epoch =  4  Train loss =  25938.7  Validation accuracy =  0.482  Test Accuracy =  0.505
Epoch =  7  Train loss =  4971.3613  Validation accuracy =  0.51000005  Test Accuracy =  0.523
Epoch =  9  Train loss =  2116.0164  Validation accuracy =  0.52199996  Test Accuracy =  0.54599994
Epoch =  10  Train loss =  1638.8682  Validation accuracy =  0.598  Test Accuracy =  0.60899997
Epoch =  11  Train loss =  637.4615  Validation accuracy =  0.642  Test Accuracy =  0.65099996
Epoch =  12  Train loss =  355.74527  Validation accuracy =  0.658  Test Accuracy =  0.67599994
Epoch =  13  Train loss =  69.256165  Validation accuracy

In [19]:
norm_deg = tf.linalg.diag(1.0 / tf.sqrt(deg))
norm_adj = tf.matmul(norm_deg,tf.matmul(adj,norm_deg))
train(features,norm_adj,gnn,32,200,0.01)

Epoch =  0  Train loss =  53709.945  Validation accuracy =  0.14799999  Test Accuracy =  0.166
Epoch =  1  Train loss =  64370.855  Validation accuracy =  0.26999998  Test Accuracy =  0.325
Epoch =  2  Train loss =  88134.59  Validation accuracy =  0.31999996  Test Accuracy =  0.346
Epoch =  3  Train loss =  87549.9  Validation accuracy =  0.482  Test Accuracy =  0.50999993
